# GINへの同期の失敗を解消する。

GINへの同期で失敗した場合は以下の各セッションをに順次実行してください。<br>
同期が失敗した場合、実行セルの出力結果に`リポジトリ側の変更と競合しました。競合を解決してください。`と表示されます。<br>
それ以外の時は、以下の処理は実行しないでください。

## 0. コンフリクトが発生しているファイルを特定する。

In [ ]:
import os
from utils import display_util
from utils.common import common
from utils.git import git_module
from IPython.display import clear_output

conflict_filepaths = git_module.get_conflict_filepaths()
annex_path_list = git_module.get_annex_content_file_paht_list()

# get 'git-annex' conflict path
annex_conflict_filepaths = common.get_AND_elements(conflict_filepaths, annex_path_list)
# get 'git' conflict path
git_conflict_filepaths = list(set(conflict_filepaths) - set(annex_conflict_filepaths))
clear_output()

In [ ]:
print(git_conflict_filepaths)
print(annex_conflict_filepaths)

In [ ]:
# get conflict variants
from utils.common import common
stdout, stderr, rt = common.exec_subprocess('git annex resolvemerge', False)
result = stdout.decode('utf-8')

from utils import display_util

if len(git_conflict_filepaths) > 0:
    display_util.display_info(msg='『1. 競合ファイル(git)を修正する。』を必ず実行してください。')
if len(annex_conflict_filepaths) > 0:
    display_util.display_info(msg='『2. 競合ファイル(git-annex)を修正する。』を必ず実行してください。')


## 1. 競合ファイル(git)を修正する。

以下のタスクを実行すると、競合が起きているファイルパスが順次、表示されます。<br>
以下の手順に従って競合ファイルを修正してください。<br>

1. 下のセルを実行する。
2. 競合ファイルのパスを確認する。
   
※ 競合が発生したファイルパスと`修正完了`ボタンが表示されます。<br>
※ この時点では、`修正完了`ボタンをクリックしないください。

![競合ファイルパス](./images/Display_of_conflicting_file_paths.png)

3. ダッシュボートの`File` > `Open...`と順にクリックする。
4. 競合が発生しているファイルにチェックを入れ、`edit`ボタンをクリックする。
5. ファイルを編集して競合を解消する。

* 編集前

![編集前](./images/Before_conflict_resolution.png)

* 編集後

![編集前](./images/After_conflict_resolution.png)

1. ダッシュボートの`File` > `Save`と順にクリックし、保存する。
2. `修正完了`ボタンをクリックする。
   1. 複数競合が発生している場合は、手順[2. 競合ファイルのパスを確認する]に戻り、次の競合ファイルの操作を行ってください。
   2. 全ての競合解消操作が完了した場合、`次にお進みください。`とメッセージ表示されます。次にお進みください。


In [ ]:
# ユーザにgit管理ファイルを手動で編集してもらう。
from ipywidgets import Button
from utils import display_util
from IPython.display import display

def on_click_callback(clicked_button: Button) -> None:
    clicked_button.button_style = 'success'
    display_util.display_info(msg="次にお進みください。")

display_util.display_msg(msg='競合が発生しているファイルは以下です。', fore=None, back=None, tag='p')
for i, git_conflict_filepath in enumerate(git_conflict_filepaths):
    display_util.display_msg(msg='{} : {}'.format(int(i) + 1, git_conflict_filepath), fore=None, back=None, tag='p')
    button = Button(description='修正完了', button_style='')
    button.on_click(on_click_callback)
    display(button)


In [ ]:
# add git content
from utils.common import common
import os
os.chdir(os.environ['HOME'])
for git_conflict_filepath in git_conflict_filepaths:
    stdout, stderr, rt = common.exec_subprocess('git add {}'.format(git_conflict_filepath), False)
    result = stdout.decode('utf-8')

## 2. 競合ファイル(git-annex)を修正する。

In [ ]:
from datalad import api
import os
from utils.git import git_module

os.chdir(os.environ['HOME'])
remote_annex_variant_paths =  git_module.get_remote_annex_variant_path(annex_conflict_filepaths)

# get information of datalad
multi_status_info = api.status(path=['experiments/'])

# create git-annex varitants dict
variants_dict = dict[str, dict]()
for conflict_annex_path in annex_conflict_filepaths:
    variants_dict[conflict_annex_path] = dict[str, str]()

for unit_status_info in multi_status_info:
    if unit_status_info['state'] == 'added':
        for conflict_annex_path in annex_conflict_filepaths:
            # adjust file path that is conflicted for extract variant paht
            dirpath = os.path.dirname(conflict_annex_path)
            filename_no_extension = os.path.splitext(os.path.basename(conflict_annex_path))[0]
            target_path = '{}/{}.variant-'.format(dirpath, filename_no_extension)

            path = unit_status_info['path']
            refds = '{}/'.format(unit_status_info['refds'])
            path = path.replace(refds, '', 1)
            if path.startswith(target_path):
                variant_list = variants_dict[conflict_annex_path]
                if path in remote_annex_variant_paths:
                    variant_list['remote'] =  path
                else:
                    variant_list['HEAD'] =  path
                variants_dict[conflict_annex_path] = variant_list
print(variants_dict)

In [ ]:
# recode conflict num
key_num = len(variants_dict.keys())

In [ ]:
import panel as pn
from ipywidgets import Button
from utils import display_util, conflict_helper_util
pn.extension()

display_util.display_msg(msg='各ファイルに対するアクションを選択し、確定してください。', fore=None, back=None, tag='p')
column = pn.Column()
form_list = list()
options = conflict_helper_util.get_annex_conflict_options()

for i, conflict_filepath in enumerate(variants_dict):
    title = pn.widgets.StaticText(name=str(int(i) + 1), value=conflict_filepath, width=700)
    form_list.append(title)
    head = pn.widgets.StaticText(name='HEAD', value=variants_dict[conflict_filepath]['HEAD'], width=700)
    form_list.append(head)
    remote = pn.widgets.StaticText(name='Remote', value=variants_dict[conflict_filepath]['remote'], width=700)
    form_list.append(remote)
    form = pn.widgets.Select(options=options, width=200)
    form_list.append(form)

# Display GUI.
for form in form_list:
    column.append(form)
display(column)

def on_click_callback(clicked_button: Button) -> None:
    clicked_button.button_style = 'success'
    display_util.display_msg(msg='選択された内容は以下になります。お間違いが無ければ次にお進みください。', fore=None, back=None, tag='p')
    display_util.display_msg(msg='もし、変更したい場合は、再度、セルを実行してください。', fore=None, back=None, tag='p')
    total = len(form_list)
    block_num = int(total/4)
    for i in range(block_num):
        start_index =i*4
        end_index = start_index+4
        block = form_list[start_index:end_index]
        v = variants_dict[block[0].value]
        v['action'] = block[3].value
        variants_dict[block[0].value] = v
        display_util.display_msg(msg='{}. {} : {}'.format(i+1, block[0].value, block[3].value), fore=None, back=None, tag='p')

button = Button(description='確定', button_style='')
button.on_click(on_click_callback)
display(button)

In [ ]:
for v in form_list:
    print(v)

In [ ]:
import json

j = json.dumps(variants_dict, indent=4)
print(j)

In [ ]:
from utils import conflict_helper_util, display_util
import os
import panel as pn
pn.extension()
## 両方残す場合、ファイル名の変更を促す
column = pn.Column()
rename_form_list = list()
if conflict_helper_util.is_more_than_both_remain(variants_dict):
    display_util.display_msg(msg='<<{}>>を選択されたファイルに対して新しいファイル名を入力してください。'.format(conflict_helper_util.get_value_BOTH_REMAIN()), fore=None, back=None, tag='p')
    display_util.display_msg(msg='ファイル名のみ入力することに注意してください。'.format(conflict_helper_util.get_value_BOTH_REMAIN()), fore=None, back=None, tag='p')

    for i, conflict_filepath in enumerate(variants_dict):
        if conflict_helper_util.is_both_remain(variants_dict[conflict_filepath]):
            # ファイル名変更フォームの生成
            title = pn.widgets.StaticText(name=str(int(i) + 1), value=conflict_filepath, width=700)
            rename_form_list.append(title)
            head = pn.widgets.StaticText(name='HEAD', value=variants_dict[conflict_filepath]['HEAD'], width=700)
            rename_form_list.append(head)
            rename_input_for_head = pn.widgets.TextInput(placeholder=os.path.basename(variants_dict[conflict_filepath]['HEAD']), width=700)
            rename_form_list.append(rename_input_for_head)
            remote = pn.widgets.StaticText(name='Remote', value=variants_dict[conflict_filepath]['remote'], width=700)
            rename_form_list.append(remote)
            rename_input_for_remote = pn.widgets.TextInput(placeholder=os.path.basename(variants_dict[conflict_filepath]['remote']), width=700)
            rename_form_list.append(rename_input_for_remote)

    # Display GUI.
    for form in rename_form_list:
        column.append(form)
    display(column)

    # settin button action
    def on_click_callback(clicked_button: Button) -> None:
        clicked_button.button_style = 'success'
        total = len(rename_form_list)
        block_num = int(total/5)
        
        for i in range(block_num):
            start_index =i*5
            end_index = start_index+5
            block = rename_form_list[start_index:end_index]
            print("=====================================================")
            for v in block:
                print(v)
            v = variants_dict[block[0].value]
            rename_dict = dict()
            rename_dict[block[1].value] = block[2].value
            rename_dict[block[3].value] = block[4].value
            v['rename'] = rename_dict
            variants_dict[block[0].value] = v
        # TODO : verify file name

        display_util.display_msg(msg='ファイル名が確定されました。もし、変更したい場合は、再度、セルを実行してください。', fore=None, back=None, tag='p')

    button = Button(description='確定', button_style='')
    button.on_click(on_click_callback)
    display(button)

In [ ]:
for v in rename_form_list:
    print(v)

In [ ]:
import json

j = json.dumps(variants_dict, indent=4)
print(j)

In [ ]:
# adjust conflicted git-annex content
from utils import conflict_helper_util, display_util

git_annex_target_path = list[str]()

for i, k in enumerate(variants_dict):
    print(f'{k}への操作')
    unit_info = variants_dict[k]
    action = unit_info['action']
    print(f'action : {action}')
    if action == conflict_helper_util.get_value_BOTH_REMAIN():
        # remain both annex variants
        rename_info = unit_info['rename']
        for i, k in enumerate(rename_info):
            future_name= rename_info[k]
            new_path = conflict_helper_util.rename_file(k, future_name)
            git_annex_target_path.append(new_path)
    elif action == conflict_helper_util.get_value_REMOTE_REMAIN():
        # remain remote contents

        # delete HEAD variants
        head_file_path = unit_info['HEAD']
        conflict_helper_util.delete_file(head_file_path)

        # rename Remote variants by base file name
        remote_file_path = unit_info['remote']
        new_path = conflict_helper_util.rename_file(remote_file_path, os.path.basename(k))
        git_annex_target_path.append(new_path)

    elif action == conflict_helper_util.get_value_HEAD_REMAIN():
        # remain HEAD content
        # rename HEAD variants by base file name
        head_file_path = unit_info['HEAD']
        new_path = conflict_helper_util.rename_file(head_file_path, os.path.basename(k))
        git_annex_target_path.append(new_path)
        # delete Remote variants
        remote_file_path = unit_info['remote']
        conflict_helper_util.delete_file(remote_file_path)
    else:
        raise Exception('不正な操作が行われました。')

## 3. GINへ修正内容をプッシュする。

In [ ]:
from utils.common import common
import os
os.chdir(os.environ['HOME'])
commit_msg = 'conflict resolution'
stdout, stderr, rt = common.exec_subprocess('git commit -m "{}"'.format(commit_msg), False)
result = stdout.decode('utf-8')

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
from IPython.display import clear_output
import traceback
# Push to GIN
try:
    utils.push()
    clear_output()
except:
    datalad_error = traceback.format_exc()
    display_util.display_err(msg='[ERROR] : '.format(datalad_error))
    display_util.display_err(msg='GIN-forkへのコミットのプッシュが失敗しました。')
else:
    display_util.display_info(msg='GIN-forkへのコミットのプッシュが成功しました。\nこれで競合解消タスクは完了です。')
    display_util.display_warm(msg='先ほどGIN-forkへの同期が失敗したNotebookに戻り、再度、セルを実行してください。')
    